In [1]:
'''
Creates training data for F13 channel predictor
'''


import numpy as np
import xarray as xr
import glob
from tqdm import tqdm
from util_funcs.L1C import scantime2datetime
from util_funcs import data2xarray, array_funcs
import geography

satellite = 'F13'
sensor = 'SSMI'

datapath = f'/edata1/archive/GPM/1C_{satellite}_V7/'

In [3]:
'''
Satellite: F13

Get data and quality check for training.

nscans: generally around 1600 & 3200
npixs: 64 & 128

Tb array will be set up as follows:
    Tbs =  [m x n], where m is the number of samples and n is the
           number of channels (features)
    1-2:   19.35 V and H
    3:     22.235 V
    4-5:   37.0 V and H
    6-9:   85.5 V and H #Double-sampled 85

Change cell to code to start from scratch.
'''

#Set up a random seed for reproducibility
np.random.seed(12)



#Get a random list of files
file_list = glob.glob(f'{datapath}*/*/1C.{satellite}.SSMI.*.HDF5'); file_list.sort()
raise
#Everything looks good before about 2008:
good_files = [ifile for ifile in file_list if int(ifile.split('/')[5]) < 801 
              and int(ifile.split('/')[5]) != 406 
              and int(ifile.split('/')[5]) != 203]

flist = np.random.choice(good_files, size=200)

tb_dim = 9
qual_dim = 2

#Loop through files and get good quality data.
for i, ifile in enumerate(tqdm(flist, desc="Processing Files")):

    #print(f'{i+1} of {len(flist)}, {ifile}')

    with xr.open_dataset(ifile, group='S1', decode_timedelta=False) as f:
        lat = f.Latitude.values
        lon = f.Longitude.values

    nscans, npixs = lat.shape
    Tbs = np.zeros([nscans,npixs,tb_dim], dtype=np.float32)
    qual = np.zeros([nscans,npixs,qual_dim], dtype=np.int32)

    with xr.open_dataset(ifile, group='S1', decode_timedelta=False) as f:
        qual[:,:,0] = f.Quality.values
        Tbs[:,:,0:2] = f.Tc[:,:,0:2].values #19V and H
        Tbs[:,:,2]   = f.Tc[:,:,2].values #22V
        Tbs[:,:,3:5] = f.Tc[:,:,3:5].values #37V and H

    with xr.open_dataset(ifile, group='S2', decode_timedelta=False) as f:
        #Even-numbered scans line up with S1 Scans (0,2,4,...)
        qual[:,:,1] = f.Quality.values[::2,::2]
        Tbs[:,:,5] = f.Tc.values[::2,::2,0] #85V, even scans, even pixs
        Tbs[:,:,6] = f.Tc.values[::2,::2,1] #85H, even scans, even pixs
        Tbs[:,:,7] = f.Tc.values[::2,1::2,0] #85V, even scans, odd pixs
        Tbs[:,:,8] = f.Tc.values[::2,1::2,1] #85H, even scans, odd pixs

    with xr.open_dataset(ifile, group='S1/ScanTime', decode_timedelta=False) as f:
        scantime = (f.Year.values, f.Month.values, f.DayOfMonth.values, f.Hour.values, f.Minute.values, f.Second.values)

    #Change scan time format from L1C to datetime format for easier use
    scantime = scantime2datetime(scantime)

    #Get only good quality data and reshape:
    goodqual = np.all(qual == 0, axis=2)
    all_bad = np.all(goodqual == False)
    if all_bad:
        print('all were bad.')
        continue
    lat = lat[goodqual]
    lon = lon[goodqual]
    scantime = scantime[np.where(goodqual)[0]]
    Tbs = Tbs[goodqual]

    #Check for NaNs (shouldn't be any if all good, but I've seen some)
    nonans = array_funcs.find_nan_rows(Tbs, return_good=True)
    Tbs = Tbs[nonans]
    lat = lat[nonans]
    lon = lon[nonans]
    scantime = scantime[nonans]
    

    #Attach GPROF surface map data to each pixel
    sfctype = geography.attach_gpm_sfctype(lat, lon, scantime, sensor=sensor)

    npixs = lat.size

    #Output as NetCDF
    dset = data2xarray(data_vars = [lat, lon, scantime, sfctype, Tbs],
                       var_names = ['latitude', 'longitude', 'scantime', 'sfctype', 'Tbs'],
                       dims = [npixs, tb_dim],
                       dim_names = ['pixels', 'channels'])


    if i == 0:
        training_dataset = dset
    else:
        training_dataset = xr.concat((training_dataset, dset), dim='pixels')

training_dataset.to_netcdf(f'training_data/{satellite}_training_data.nc', engine='netcdf4')



RuntimeError: No active exception to reraise

In [2]:
with xr.open_dataset(f'training_data/{satellite}_training_data.nc') as f:
    sfctype = f.sfctype.values

for i in range(19):
    print(f'Number of pixels with surface type {i}: {np.sum(sfctype == i)}')

print(f'Total number of ocean pixels: {np.sum(sfctype == 1)}')
print(f'Total number of nonocean pixels: {np.sum(sfctype > 1)}')

Number of pixels with surface type 0: 0
Number of pixels with surface type 1: 10264598
Number of pixels with surface type 2: 2130464
Number of pixels with surface type 3: 633839
Number of pixels with surface type 4: 400243
Number of pixels with surface type 5: 322896
Number of pixels with surface type 6: 185121
Number of pixels with surface type 7: 338949
Number of pixels with surface type 8: 110586
Number of pixels with surface type 9: 140473
Number of pixels with surface type 10: 581777
Number of pixels with surface type 11: 1960635
Number of pixels with surface type 12: 302901
Number of pixels with surface type 13: 434416
Number of pixels with surface type 14: 488495
Number of pixels with surface type 15: 335350
Number of pixels with surface type 16: 98928
Number of pixels with surface type 17: 789595
Number of pixels with surface type 18: 430695
Total number of ocean pixels: 10264598
Total number of nonocean pixels: 9685363


In [4]:
file_list

['/edata1/archive/GPM/1C_F13_V7/0001/000101/1C.F13.SSMI.XCAL2018-V.20000101-S004616-E022812.024619.V07A.HDF5',
 '/edata1/archive/GPM/1C_F13_V7/0001/000101/1C.F13.SSMI.XCAL2018-V.20000101-S022813-E041009.024620.V07A.HDF5',
 '/edata1/archive/GPM/1C_F13_V7/0001/000101/1C.F13.SSMI.XCAL2018-V.20000101-S041010-E055206.024621.V07A.HDF5',
 '/edata1/archive/GPM/1C_F13_V7/0001/000101/1C.F13.SSMI.XCAL2018-V.20000101-S055207-E073404.024622.V07A.HDF5',
 '/edata1/archive/GPM/1C_F13_V7/0001/000101/1C.F13.SSMI.XCAL2018-V.20000101-S073405-E091601.024623.V07A.HDF5',
 '/edata1/archive/GPM/1C_F13_V7/0001/000101/1C.F13.SSMI.XCAL2018-V.20000101-S091602-E105758.024624.V07A.HDF5',
 '/edata1/archive/GPM/1C_F13_V7/0001/000101/1C.F13.SSMI.XCAL2018-V.20000101-S105759-E123955.024625.V07A.HDF5',
 '/edata1/archive/GPM/1C_F13_V7/0001/000101/1C.F13.SSMI.XCAL2018-V.20000101-S123956-E142152.024626.V07A.HDF5',
 '/edata1/archive/GPM/1C_F13_V7/0001/000101/1C.F13.SSMI.XCAL2018-V.20000101-S142153-E160350.024627.V07A.HDF5',
 